In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import matplotlib.patches as mpatches

# Initialize an empty list to store all CSV file paths
file_paths = []
date_objects = []
# Iterate over the range of years
for year in range(2010, 2023):
    yr = str(year)

    # Construct the path for each year
    folder_path = os.path.join('/efs/jrclark/PFe Data/PFe_Catalogs/S PFe Characteristics', f'{yr}_03')
    
    # Check if the directory exists
    if os.path.exists(folder_path):
        # Get the list of files in the directory
        files = os.listdir(folder_path)
        
        # Iterate over each file in the directory
        for file in files:
            filename = str(file)
            #print(filename)
            #subfilename=filename[12:25]
            #print(subfilename)
            #date_time_obj = datetime.strptime(subfilename, '%Y%m%dT%H%M')
            #print(date_time_obj)
            # Check if the file is a CSV file
            if file.endswith('.csv'):
                # Construct the full path of the CSV file
                full_path = os.path.join(folder_path, file)
                # Append the full path to the file_paths list
                subfilename=filename[12:25]
                date_time_obj = datetime.strptime(subfilename, '%Y%m%dT%H%M')
                #print(date_time_obj)
                file_paths.append(full_path)
                date_objects.append(date_time_obj)

    else:
        print(f"Directory {folder_path} does not exist")

# Print the list of all CSV file paths
#print(file_paths)
file_paths.sort()
#print(file_paths)
#print(len(file_paths))
date_objects.sort()
#print(date_objects)
#print(date_time_obj)
        
# Iterate over the list of CSV file paths
for i, x in enumerate(file_paths):
    faculaeframe = pd.read_csv(x)
    extracted_coords = faculaeframe.sort_values(by='Area', ascending=False)
    data_cond = extracted_coords[['X_Centroid', 'Y_Centroid', 'Lat', 'Lon', 'Area', 'Avg_Br']]
    data_cond2 = extracted_coords[['Area']]
    data_cond3 = extracted_coords[['Avg_Br']]
    #print(data_cond)
    #print(data_cond2)
    #print(extracted_coords)
  
    # Select specific columns from the filtered DataFrame
    #print(extracted_coords[['X_Centroid', 'Y_Centroid', 'Lat', 'Lon','Area']])
    all_areas_list = data_cond2['Area'].tolist()
    all_bfield_list = data_cond3['Avg_Br'].tolist()
    #print(f' areas list {all_areas_list}')
    #print(f'b field list {all_bfield_list}')
    
    marker_size=[]
    for j in all_areas_list:
        if ((j >0.0) & (j<=10.0)):
            a=10
        elif ((j >10.0) & (j<=20.0)):
            a=15
        elif ((j >20.0 ) & (j<=30.0)):
            a=20
        else:
            a=25
        marker_size.append(a)
    #print(f' marker size {marker_size}')
    
    color_symbol=[]
    #color_label=['Blue','Caribbean Blue', 'Charlotte','Yellow', 'Orange', 'Red']
    for k in all_bfield_list:
        if (k <= -400):
            b = 'b'
            color_lbl='Blue'
        elif ((k > -400) & (k<=-200)):
            b = '#17BECF'
            color_lbl='Caribbean Blue'
        elif ((k > -200 ) & (k<=0)):
            b = '#9EDAE5'
            color_lbl='Charlotte'
        elif ((k > 0 ) & (k<=200)):
            b = '#dede00'
            color_lbl='Yellow'
        elif ((k > 200 ) & (k<=400)):
            b = '#ff7f00'
            color_lbl='Orange'
        else:
            b = 'r'
            color_lbl='Red'
        color_symbol.append(b)
        #color_label.append(color_lbl)
    #print(f' Color symbol {color_symbol}')
    
    x_centroid_list = data_cond['X_Centroid'].tolist()
    y_centroid_list = data_cond['Y_Centroid'].tolist()
   # print(f' x centroid {x_centroid_list}')
   # print(f' y centroid {y_centroid_list}')
    
    lat_list = data_cond['Lat'].tolist()
    lon_list = data_cond['Lon'].tolist()
    #print(f' lat list {lat_list}')
    #print(f' lon list {lon_list}')
    
    latitude1 = [-80] * 361
    latitude2 = [-70] * 361
    longitude = list(range(0, 361))
    long_arry = np.array(longitude)
    lat80_arr = np.array(latitude1)
    lat70_arr = np.array(latitude2)
    
    def toxy_coord(long_ln, lat_ln):
        theta = long_ln * (math.pi / 180.0)
        lat_in_rad = lat_ln * (math.pi / 180.0)

        R = 1
        rho = R * ((math.pi / 2) + lat_in_rad)
        x_long = rho * np.sin(theta)  # long
        y_lat = -rho * np.cos(theta)  # lat
        return x_long, y_lat
    
    x_80, y_80 = toxy_coord(long_arry, lat80_arr)
    x_70, y_70 = toxy_coord(long_arry, lat70_arr)
    
    x_arr, y_arr = toxy_coord(np.array(lon_list), np.array(lat_list))
    
     # Initialize plot
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Scatter plot of latitude and longitude
    lat_plot = ax.plot(x_80, y_80, color='r' )
    lat_plot2 = ax.plot(x_70, y_70, color='b')
    
    
    
    x_0 = ax.axhline(0, color='black', linewidth=1)  # x = 0 line
    y_0 = ax.axvline(0, color='black', linewidth=1)  # x = 0 line
    
    scatter = ax.scatter(x_arr, y_arr,marker_size, color_symbol, marker='o')

    # produce a legend with a cross-section of sizes from the scatter
    size_labels= ['A = (0, 10]','A = (10, 20]', 'A = (20, 30]', 'A > 30']
    size_handles, _ = scatter.legend_elements(prop="sizes", alpha=0.6)
    
    color_labels = ['B <=-400', 'B = (-400, -200]', 'B = (-200, 0]', 
                    'B = (0, 200]', 'B = (200, 400]', 'B > 400']
    color_symbol = ['blue', '#17BECF','#9EDAE5','#dede00', '#ff7f00', 'r']
    # Create custom color handles
    color_handles = [mpatches.Patch(color=color_symbol[i], label=color_labels[i]) for i in range(len(color_labels))]

    #Add color and size legend to the plot
    legend1 = ax.legend(color_handles, color_labels, loc="upper left", title="Colors", fontsize=10)
    legend2 = ax.legend(size_handles, size_labels, loc="upper right", title="Sizes", fontsize= 10)
    ax.add_artist(legend1)
    ax.add_artist(legend2)
    
    
    # Plot the plus sign at the intersection with increased size
    intersect=ax.plot(0, 0, '+', color='#FF7F0E', markersize=12)
    
    ax.set_xlim(-5, 5)
    ax.set_xlim(-5, 5)
    
    plt.title(f' Azimuthal Equidistant Projection of PFe {date_objects[i]}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axis('equal')
    plt.tight_layout()
    
    plt.text(0.02, 0.32, '-70°')
    plt.text(0.02, 0.15, '-80°')
    
    # Remove the top and right spines (borders)
    ax = plt.gca()
    ax.grid(False, which='both')
    
    # Saving the plots
    #plt.show()
    plt.savefig('/efs/okome7/movie_images/2011-2022_March_PFe/Frame_'+ str(date_objects[i])+'.png',bbox_inches = 'tight')
    plt.close()
    

Directory /efs/jrclark/PFe Data/PFe_Catalogs/S PFe Characteristics/2010_03 does not exist


In [2]:
import os
import zipfile
from IPython.display import FileLink

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

folder_to_zip = '/efs/okome7/movie_images/2011-2022_March_PFe'  # Replace with your folder name
output_zip = '2011-2022_March_PFe.zip'  # Replace with your desired output zip file name

zip_folder(folder_to_zip, output_zip)

FileLink(output_zip)

/home/jovyan/efs/okome7/2011-2022_March_PFe.zip